In [25]:
from HMM_Functions import *
from matplotlib import cm, pyplot as plt
import seaborn
from matplotlib.dates import YearLocator, MonthLocator, DayLocator
import pandas as pd
import numpy as np
from datetime import datetime

In [26]:
tpm = [[0.3, 0.3, 0.2, 0.1, 0.05, 0.05],
       [0.25, 0.3, 0.25, 0.1, 0.05, 0.05],
       [0.1, 0.1, 0.3, 0.2, 0.2, 0.1],
       [0.1, 0.1, 0.2, 0.3, 0.2, 0.1],
       [0.05, 0.05, 0.1, 0.25, 0.3, 0.25],
       [0.05, 0.05, 0.1, 0.2, 0.3, 0.3]]

#tpm = create_TPM(6)

pi = [.2, .1, .1, .2, .1, .1]

epm = create_EPM(6, 2)
emissions = ['Decreasing', 'Decreasing','Increasing','Decreasing',
 'Decreasing','Decreasing','Decreasing', 'Decreasing', 'Increasing','Decreasing','Decreasing',
 'Decreasing','Increasing','Increasing','Decreasing','Decreasing']

In [42]:
'''
                            TPM          FWD PROB
P(Xt+k+1 | e1:t) = ∑t+k P(Xt+k+1|xt+k) P(xt+k|e1:t)


∑t+k P(Xt+k+1|xt+k) * P(xt+k|e1:t): Iterate through ALL the states and add up the TPM * most recent fwd probability

Outcome: matrix with probabilities for each of hte next states, prediction is the highest one:
          I    D
state 0: .05  .05  = .1  -> .5 .5 = 1
state 1: .05  .05  = .1  -> .5 .5 = 1
state 2: .1   .1  = .2   
state 3: .05  .25 = .3   -> .05 .95 = 1
state 4: .1.  .1  = .2
state 5: .05  .05 =.01

'''

def filtering_K(tpm, epm, emissions, pi):

    t = len(emissions)
    NUM_STATES = len(tpm)
    prediction = []
    fwrd = forward(tpm, epm, pi, emissions)
    pred_list = []
    
    #starting_fwd = fwd[T][state_i]
    #starting_fwd
    
    #∑t+k P(Xt+k+1|xt+k) * P(xt+k|e1:t): Iterate through ALL the states and add up the TPM * most recent fwd probability
    
    '''
    
   do we need to do this part?  
   EPM of I at state 0  * probability state 0| KNOW that we were just in 0 *  fwd of being in state 0 at previous time step
   EPM of D at state 0  * probability state 0| KNOW that we were just in 0 *  fwd of being in state 0 at previous time step
   
          probability state 0| KNOW that we were just in 1 *  fwd of state 1
          probability state 0| KNOW that we were just in 2 *  fwd of state 2
          probability state 0| KNOW that we were just in 3 *  fwd of state 3
          probability state 0| KNOW that we were just in 4 *  fwd of state 4
          probability state 0| KNOW that we were just in 5 *  fwd of state 5
            add all this up to get the probabiltiy of next state being state 0
            
          probability state 1| KNOW that we were just in 0 *  fwd of state 0
          probability state 1| KNOW that we were just in 1 *  fwd of state 1
          probability state 1| KNOW that we were just in 2 *  fwd of state 2
          probability state 1| KNOW that we were just in 3 *  fwd of state 3
          probability state 1| KNOW that we were just in 4 *  fwd of state 4
          probability state 1| KNOW that we were just in 5 *  fwd of state 5
            add all this up to get the probabiltiy of next state being state 1
            
          probability state 2| KNOW that we were just in 0 *  fwd of state 0
          probability state 2| KNOW that we were just in 1 *  fwd of state 1
          probability state 2| KNOW that we were just in 2 *  fwd of state 2
          probability state 2| KNOW that we were just in 3 *  fwd of state 3
          probability state 2| KNOW that we were just in 4 *  fwd of state 4
          probability state 2| KNOW that we were just in 5 *  fwd of state 5
            add all this up to get the probabiltiy of next state being state 2
        '''
        
    for state_i in range(NUM_STATES):
        pred_state_dist = tpm[state_i]
        total = 0

        for state_j in range(NUM_STATES):
            transition_prob = pred_state_dist[state_j]
            fwd_prob =  fwrd[t-1][state_j]
            
            total = total + (transition_prob * fwd_prob)
                
        pred_list.append(total)
        
    denominator = 0
    normalized_list = []
    for i in range(len(pred_list)):
        denominator += pred_list[i]
        
    print('denom is...', denominator)
    
    for i in range(len(pred_list)):
        value = pred_list[i]/denominator
        normalized_list.append(value)
        
    
    print('normalized_list is..', normalized_list)
    
    checksum = sum(normalized_list)
    
    print('equals 1?',checksum)
    
    maxstate = max(normalized_list)
    print(maxstate)
    
    return(pred_list)
            
    

In [43]:
pred = filtering_K(tpm, epm, emissions, pi)
pred

denom is... 1.2500170657035237e-05
normalized_list is.. [0.15173029775290853, 0.15479591185558508, 0.17400477849233528, 0.17511455356913824, 0.1736105761273977, 0.17074388220263512]
equals 1? 1.0
0.17511455356913824


[1.8966546157541268e-06,
 1.9349753152061975e-06,
 2.1750894262938054e-06,
 2.188961804144767e-06,
 2.170161829458679e-06,
 2.1343276661776603e-06]

In [3]:

def filtering(tpm, epm, pi, emissions):
    predict = []
    
    T = len(emissions)
    NUM_STATES = len(tpm)
    
    forwrd = forward(tpm, epm, pi, emissions[:-1])
    bkwrd = backward(tpm, epm, pi, emissions)
    
    
    normalization_const = #something
    
    for state_i in range(NUM_STATES):
        a = bkwrd[T][state_i]

        b = 0
        for state_j in range(NUM_STATES):
            b += tpm[state_j][state_j+1] * forwrd[T-1][state_j]
        
        predict.append(normalization_const * a * b)
    
    return predict

SyntaxError: invalid syntax (<ipython-input-3-15a688e2bd0d>, line 11)